In [1]:
import sapien.core as sapien
from Tools import misc, ForwardKinematics, ModelSim, NumForwardDynamicsDer, Dynamics, MathForwardDynamicsDer
import numpy as onp
import jax.numpy as np
from jax import jit, jacfwd, jacrev, grad
from ilqr import ILQR
from datetime import datetime
from tqdm.notebook import trange
import matplotlib.pyplot as plt

%matplotlib notebook
%env XLA_PYTHON_CLIENT_MEM_FRACTION=.40


sim = sapien.Engine()
renderer = sapien.OptifuserRenderer()
sim.set_renderer(renderer)
render_controller = sapien.OptifuserController(renderer)

Using default glsl path /home/zack/anaconda3/envs/ml/lib/python3.7/site-packages/sapien/glsl_shader/130
env: XLA_PYTHON_CLIENT_MEM_FRACTION=.40


In [2]:
DEBUG = False

stabled = False


def create_scene(timestep, visual):
    s = sim.create_scene([0, 0, 0])
    s.add_ground(-1)
    s.set_timestep(timestep)

    loader = s.create_urdf_loader()
    loader.fix_root_link = True
    if visual:
        loader.collision_is_visual = True
        s.set_ambient_light([0.5, 0.5, 0.5])
        s.set_shadow_light([0, 1, -1], [0.5, 0.5, 0.5])

    # build
    robot = loader.load("../assets/Arm/panda.urdf")

    for joint in robot.get_joints():
        joint.set_drive_property(stiffness=0, damping=1)
    robot.set_qpos(np.array([0, 1.7, 0, -1.5, 0, 2.5, 0.7, 0.4, 0.4]))

    return s, robot


sim_timestep = 1 / 60
optim_timestep = 1 / 60
s0, robot = create_scene(sim_timestep, True)

render_controller.set_camera_position(-5, 0, 0)
render_controller.set_current_scene(s0)

In [19]:
m1 = robot.compute_mass_matrix()
for _ in range(5):
    u = onp.random.random(robot.dof) * 500
    robot.set_qf(u)
    s0.step()

    m2 = robot.compute_mass_matrix()
    print(np.linalg.norm(m2-m1))
    m1 = m2

1.6421484
1.4442369
0.6969862
2.322577
2.547936


In [ ]:
def inverseDynamics(robot:sapien.Articulation, qAcc):
    # linear
    M = robot.compute_mass_matrix()
    linear_force = M @ qAcc

    # other
    F = robot.compute_passive_force(False, False, False)

    return linear_force + F

In [ ]:
packs = []

for _ in range(100):
    qAcc = onp.random.randn(robot.dof)
    robot.set_qf(qAcc)
    s0.step()

    f1 = robot.compute_inverse_dynamics(qAcc)
    f2 = inverseDynamics(robot, qAcc)

    diff = onp.linalg.norm(f1 - f2)
    if diff > 1e-6:
        print(diff)

In [ ]:
dym = Dynamics(robot, sim_timestep, True, True, True)

for _ in range(5):
    qpos = robot.get_qpos()

    u = onp.random.randn(robot.dof) * 10
    robot.set_qf(u)
    s0.step()

    dPos = dym.forward(u)

    pos1 = robot.get_qpos()
    pos2 = qpos + dPos

    diff = np.linalg.norm(pos1 - pos2)

    # print(diff)

    print(pos1)
    print(pos2, "\n")

In [ ]:
fu = jacfwd(dym.forward)
u = onp.random.randn(robot.dof) * 10
fu(u)

In [ ]:
der = MathForwardDynamicsDer(robot, sim_timestep, True, True, True)

der.fu(u, None).shape